# Data Processing for Climate Downscaling

ClimateLearn makes it super easy to prepare data for your machine learning pipelines. In this tutorial, we'll see how to download [ERA5](https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5) data from [WeatherBench](https://github.com/pangeo-data/WeatherBench) and prepare it for both the forecasting and [downscaling](https://uaf-snap.org/how-do-we-do-it/downscaling) tasks. This tutorial is intended for use in Google Colab.

## Google Colab setup
You might need to restart the kernel after installing ClimateLearn so that your Colab environment knows to use the correct package versions.

In [ ]:
!pip install climate-learn

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download

In the [downscaling task](https://uaf-snap.org/how-do-we-do-it/downscaling), we want to build a machine learning model that can map low-resolution weather patterns (source) to high-resolution weather patterns (target). Here, let's download two datasets for `2m_temperature` at 5.625 degrees resolution and at 2.8125 degrees resolution respectively.

The following cell will take several minutes to run - the scale of climate data is huge!

In [ ]:
from climate_learn.data import download

root = "/content/drive/MyDrive/ClimateLearn"
source = "weatherbench"
dataset = "era5"
low_resolution = "5.625"
hi_resolution = "2.8125"
variable = "2m_temperature"
years = range(1979, 2018)

download(root=root, source=source, dataset=dataset, resolution=low_resolution, variable=variable)
download(root=root, source=source, dataset=dataset, resolution=hi_resolution, variable=variable)

ClimateLearn comes with some utilities to view the downloaded data in its raw format. This can be useful as a quick sanity check that you have the data you expect. Climate data is natively stored in the [NetCDF format](https://www.unidata.ucar.edu/software/netcdf/), which means it comes bundled with lots of helpful named metadata such as latitude, longitude, and time. However, we want the data in a form that can be easily ingested by PyTorch machine learning models.

In [ ]:
from climate_learn.utils.data import load_dataset, view

my_dataset = load_dataset(f"{root}/data/{source}/{dataset}/{low_resolution}/{variable}")
view(my_dataset)

## Preparing data for downscaling

Next, we specify the dataset arguments. This is the same procedure as for forecasting, but with two datasets now: one set of arguments is for the source, and another set of arguments is for the target.

In [ ]:
from climate_learn.data.climate_dataset.args import ERA5Args

lowres_data_args = ERA5Args(
    root_dir=f"{root}/data/{source}/{dataset}/{low_resolution}/",
    variables=[variable],
    years=years
)

highres_data_args = ERA5Args(
    root_dir=f"{root}/data/{source}/{dataset}/{hi_resolution}",
    variables=[variable],
    years=years
)

Then, we specify the task arguments.

In [ ]:
from climate_learn.data.tasks.args import DownscalingArgs

downscaling_args = DownscalingArgs(
    dataset_args=lowres_data_args,
    highres_dataset_args=highres_data_args,
    in_vars=[variable],
    out_vars=[variable],
    subsample=6,
)

Finally, we specify the data module, which looks the same as for the forecasting task.

In [ ]:
from climate_learn.data import DataModuleArgs, DataModule

data_module_args = DataModuleArgs(
    task_args=downscaling_args,
    train_start_year=1979,
    val_start_year=2015,
    test_start_year=2017,
    end_year=2018
)

data_module = DataModule(
    data_module_args=data_module_args,
    batch_size=128,
    num_workers=1
)

Congralutions! Now you know how to load and process data with ClimateLearn. Please visit our [docs](https://climatelearn.readthedocs.io/en/latest/user-guide/datasets.html) to learn more.